## ID3 algorithm

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [70]:
def split(array):
    dic={}
    for i in np.unique(array):
        dic.update({i:np.where(array==i)[0]})
    return dic

In [71]:
print(split(np.array([0,1,2])))
print(split(np.array([1,0,1,0,0,1,0])))
print(split(np.array([1,0,3,2,0,1,1])))

{0: array([0]), 1: array([1]), 2: array([2])}
{0: array([1, 3, 4, 6]), 1: array([0, 2, 5])}
{0: array([1, 4]), 1: array([0, 5, 6]), 2: array([3]), 3: array([2])}


In [72]:
import math
def entropy(array):
    b_list=[]
    for i in np.unique(array):
        p=len(np.where(array==i)[0])/len(array)
        b_list.append(p*math.log2(p))
    return -sum(b_list)

In [73]:
print(round(entropy(np.array([0,1,0,1,1,0])),4))
print(round(entropy(np.array([1,2])),4))
print(round(entropy(np.array([1,1])),4))
print(round(entropy(np.array([1,0,0,0,0,0,0,0,0,0,0])),4))
print(round(entropy(np.array([0,0,0])),4))
print(round(entropy(np.array([1,1,1,0,1,4,4,2,1])),4))

1.0
1.0
-0.0
0.4395
-0.0
1.6577


In [74]:
def IG(x,y):
    parent_entropy=entropy(y)
    split_dict=[split(x).get(k) for k in np.unique(x)]
    for i in split_dict:
        freq=len(x[[k for k in i]])/len(x)
        child_entropy=freq*entropy(y[[k for k in i]])
        parent_entropy=parent_entropy-child_entropy
    return parent_entropy

In [75]:
x=np.array([0,1,0,1,0,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,0,1,1,2,2])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,1,0,1,2,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))

0.4591
0.2516
0.9183


In [76]:
def make_tree(X,y,attribute):
    if y.shape[0]==1 or y.shape[0]==0:
        return y

    gains=[]
    if len(X.T)==1:
        gain=IG(X.T,y)
        if (gain<=1e-05):
            return y
        gains.append(gain)
    else:
        for x in X.T:
            gain=IG(x,y)

            if (gain<=1e-05):
                return y

            gains.append(gain)
    #print(gains)
    best_feature=np.argmax(gains)
    #print(best_feature)
    results={}
    
    subset_dict=split(X[:,best_feature])
    #print(subset_dict)
    for feature_value,train_example_indices in subset_dict.items():
        #print(train_example_indices)
        child_x_subset=np.delete(X[train_example_indices],best_feature,1)
        child_y_subset=y[train_example_indices]
        child_attribute=attribute[attribute != attribute[best_feature]]
        #print(child_x_subset)

        results["%s = %s" %(attribute[best_feature], feature_value)] = make_tree(child_x_subset, child_y_subset,child_attribute)

    return results

In [77]:
x=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].values
y=pd.read_csv("dataset/tennis.csv").iloc[:,-1].values
attribute=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].columns.values

![img](decision_img.png)

In [86]:
golf_df=pd.read_csv("dataset/tennis.csv").iloc[:,1:]
golf_df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [78]:
print(attribute)
print(x)
print("label\n",y)
print("decision tree:\n",make_tree(x,y,attribute))

['outlook' 'temp' 'humidity' 'wind']
[['Sunny' 'Hot' 'High' 'Weak']
 ['Sunny' 'Hot' 'High' 'Strong']
 ['Overcast' 'Hot' 'High' 'Weak']
 ['Rain' 'Mild' 'High' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Strong']
 ['Overcast' 'Cool' 'Normal' 'Strong']
 ['Sunny' 'Mild' 'High' 'Weak']
 ['Sunny' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Mild' 'Normal' 'Weak']
 ['Sunny' 'Mild' 'Normal' 'Strong']
 ['Overcast' 'Mild' 'High' 'Strong']
 ['Overcast' 'Hot' 'Normal' 'Weak']
 ['Rain' 'Mild' 'High' 'Strong']]
label
 ['No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes'
 'No']
decision tree:
 {'outlook = Overcast': array(['Yes', 'Yes', 'Yes', 'Yes'], dtype=object), 'outlook = Rain': {'wind = Strong': array(['No', 'No'], dtype=object), 'wind = Weak': array(['Yes', 'Yes', 'Yes'], dtype=object)}, 'outlook = Sunny': {'humidity = High': array(['No', 'No', 'No'], dtype=object), 'humidity = Normal': array(['Yes', 'Yes'], dtype=object)}}


In [79]:
def _traverse(x,d,attribute):
    if isinstance(d,np.ndarray):
        return d
    for key in d:
        name,value=key.split("=")
        feature_idx=attribute.tolist().index(name.strip())
        if x[feature_idx]==value.strip():
            return _traverse(x,d[key],attribute)

In [82]:

_traverse(np.array(['Rain','Mild','High','Weak']),make_tree(x,y,attribute),attribute)

array(['Yes', 'Yes', 'Yes'], dtype=object)

the answer is "Yes"

In [85]:
_traverse(np.array(['Overcast','Cool','High','Weak']),make_tree(x,y,attribute),attribute)

array(['Yes', 'Yes', 'Yes', 'Yes'], dtype=object)

the answer is "Yes"

In [96]:
for i,j in zip(golf_df.iloc[:,:-1].values,golf_df['play']):
    print("predict:",_traverse(i,make_tree(x,y,attribute),attribute))
    print('lable:',j)

predict: ['No' 'No' 'No']
lable: No
predict: ['No' 'No' 'No']
lable: No
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No']
lable: No
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No' 'No']
lable: No
predict: ['Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No']
lable: No


In conclusion: all prediction is correct

# reference
https://towardsdatascience.com/entropy-how-decision-trees-make-decisions-2946b9c18c8

---

# Building Trees using scikit-learn

## Introduction

In this lesson, we shall cover decision trees (for classification) in python, using scikit-learn and pandas. The emphasis will be on the basics and understanding the resulting decision tree. Scikit-Learn provides a consisitent interface for running different classifiers/regressors. For classification tasks, evaluation is performed using the same measures as we have seen before. Let's look at our example from earlier lessons and grow a tree to find our solution. 

## Objectives
You will be able to:

- Using `pandas` to prepare the data for the scikit-learn decision tree algorithm
- Train the classifier with a training dataset and evaluate performance using different measures
- Visualize the decision tree and interpret the visualization

## Import Necessary Libraries

In order to prepare data, train, evaluate and visualize a decision tree , we would need a number of packages in python. Here are the packages that you would normally consider importing before moving on. Run the cell below to import everything we'll need for this lesson. 

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import pandas as pd 
import numpy as np 

## Create Dataframe

The play tennis dataset is available in the repo as `tennis.csv`.  For this step, we'll start by importing the csv file as a pandas dataframe. Then, since all of our data is currently categorical (recall that each column is in string format), we need to encode them as numbers. For this, we'll use a handy helper objects from sklearn's `preprocessing` module. Since our target, `play`, is in a binary format, we'll use `LabelEncoder`. Since our predictors are not binary, we'll instead use `OneHotEncoder` for them. Finally, we'll print the shape of each piece of transformed data in order to make sure that it all looks correct. 
- Apply labels to target variable such that `yes=1` and `no=0`
- Apply one hot encoding to the feature set, creating ten features (outlook x 3, temp x 3, humidity x 2 , wind x 2) 
- Print the resulting features and check shape

In [102]:
# Load the dataset
df = pd.read_csv('dataset/tennis.csv') 

# Create label encoder instance
lb = LabelEncoder() 

# Create Numerical labels for classes
df['play_'] = lb.fit_transform(df['play'] ) 
df['outlook_'] = lb.fit_transform(df['outlook']) 
df['temp_'] = lb.fit_transform(df['temp'] ) 
df['humidity_'] = lb.fit_transform(df['humidity'] ) 
df['windy_'] = lb.fit_transform(df['wind'] ) 

# Split features and target variable
X = df[['outlook_', 'temp_', 'humidity_', 'windy_']] 
Y = df['play_']

# Instantiate a one hot encoder
enc = OneHotEncoder()

# Fit the feature set X
enc.fit(X)

# Transform X to onehot array 
onehotX = enc.transform(X).toarray()

onehotX, onehotX.shape, X.shape

/Users/anh/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(array([[0., 0., 1., 0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1., 1., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 0., 1., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 1., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 1., 1., 0., 1., 0.]]), (14, 10), (14, 4))

## Create Test and Training sets

Our data is now encoded properly, but we're still not ready for training. Before we do anything with a Decision Tree model, we'll want to split our data into **_training_** and **_testing_** sets.  We'll accomplish this by passing `onehotX` and `Y` to the `train_test_split` function to create a 70/30 train test split. 

In [103]:
X_train, X_test , y_train,y_test = train_test_split(onehotX, Y, test_size = 0.3, random_state = 42) 

## Train the Decision Tree 

One awesome feature of scikit-learn is the uniformity of its interfaces for every classifier--no matter what classifier we're using, we can expect it to have the same important methods such as `.fit()` and `.predict()`. This means that this next part will probably feel a little familiar.

We'll first create an instance of the classifier with any parameter values, and then we'll fit our data to the model using `.fit()` and make predictions with `X_test` using `.predict()`. 

In [104]:
clf= DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train,y_train) 
y_pred = clf.predict(X_test)

## Evaluate the Predictive Performance

Now that we have a trained model and we've generated some predictions, we cango on and see how accurate our predictions are. We can use a simple accuracy measure, AUC, a Confusion matrix, or all of them. This step is performed in the exactly the same manner , doesnt matter which  classifier you are dealing with. 

In [105]:
acc = accuracy_score(y_test,y_pred) * 100
print("Accuracy is :{0}".format(acc))

# Check the AUC for predictions
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("\nAUC is :{0}".format(round(roc_auc,2)))

# Create and print a confusion matrix 
print('\nConfusion Matrix')
print('----------------')
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy is :60.0

AUC is :0.58

Confusion Matrix
----------------


Predicted,0,1,All
True,,,
0,1,1,2
1,1,2,3
All,2,3,5


## Summary 

In this lesson, we looked at how to grow a decision tree in scikitlearn and python. We looked at different stages of data processing, training and evaluation that you would normally come across while growing a tree or training any other such classifier. We shall now move to a lab, where you will be required to build a tree for a given problem, following the steps shown in this lesson.